In [7]:
import tradingeconomics as te
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
te.login('jupyter:jupyter') # Signup for a free API key at https://developer.tradingeconomics.com/

'You are logged in as a jupyter:jupyter'

In [18]:
raw_steel = te.fetchMarkets(symbol=['JBP:COM'], initDate='2016-01-01', endDate='2019-12-31' , output_type='df')
raw_exports = te.getCmtHistorical(symbol = 'PRTWLDXX722', output_type='df')
raw_exports.tail()

,symbol,date,value
26,PRTWLDXX722,2016-12-31T00:00:00,1.178638e+09
27,PRTWLDXX722,2017-12-31T00:00:00,1.532346e+09
28,PRTWLDXX722,2018-12-31T00:00:00,1.856112e+09
29,PRTWLDXX722,2019-12-31T00:00:00,1.503772e+09
30,PRTWLDXX722,2020-12-31T00:00:00,1.324996e+09


In [17]:
raw_imports = te.getCmtHistorical(symbol = 'PRTWLDXX721', output_type='df')
raw_imports.tail()

,symbol,date,value
26,PRTWLDXX721,2016-12-31T00:00:00,1.964817e+09
27,PRTWLDXX721,2017-12-31T00:00:00,2.681680e+09
28,PRTWLDXX721,2018-12-31T00:00:00,3.296089e+09
29,PRTWLDXX721,2019-12-31T00:00:00,2.738350e+09
30,PRTWLDXX721,2020-12-31T00:00:00,2.395562e+09


In [19]:
steel = raw_steel[['close']].rename(columns={'close': 'Steel'})
steel.index.name = 'Date'

imports = raw_imports[['value', 'date']].rename(columns={'value': 'imports'})
imports['Date'] = pd.DatetimeIndex(imports['date'])
imports.set_index('Date', inplace=True)
del imports['date']

exports = raw_exports[['value', 'date']].rename(columns={'value': 'exports'})
exports['Date'] = pd.DatetimeIndex(exports['date'])
exports.set_index('Date', inplace=True)
del exports['date']

df = pd.merge(imports, exports, on ='Date')
df = df[df.index > '2016-01-31']

steel_yearly_avg = steel['Steel'].resample('Y').mean()
steel_yearly_avg=steel_yearly_avg.to_frame()
steel_yearly_avg.tail()



,Steel
Date,
2016-12-31,2339.622951
2017-12-31,3776.807377
2018-12-31,4127.468254
2019-12-31,3825.249042


In [20]:
color_dict = {'imports': 'lightcoral', 'exports': 'lightskyblue'}

label = df.index.year
Imports = df['imports'].values.tolist()
Exports = df['exports'].values.tolist()
steel = steel_yearly_avg['Steel'].values.tolist()

width = .35

data = pd.DataFrame({
 'Portugal Imports (USD)' : Imports,
 'Portugal Exports (USD)' : Exports,
 'Steel AVG Price (CNY/T)' : steel})


data[['Portugal Imports (USD)','Portugal Exports (USD)']].plot(kind='bar', width = width,figsize=(18, 12), color=[color_dict.get(x, '#333333') for x in df.columns])
data['Steel AVG Price (CNY/T)'].plot(secondary_y=True, color='black',legend='s')
ax = plt.gca()

ax.set_title('Portugal Steel and Iron Trades VS Average Annual Steel Price', color='black', size=20)
plt.xlim([-width, len(data['Portugal Imports (USD)'])-width])

plt.ylabel('Steel AVG Price (CNY/T)', fontsize=13)
ax.set_xticklabels((label))

plt.grid()
plt.show()


ValueError: arrays must all be same length